In [1]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [2]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [26]:
def policy_evaluation(env, discount_factor, theta, V, policy):
    while True:
        delta = 0
        V1 = np.copy(V)
        for id_s, state in enumerate(V):
            lst = [(env.P[id_s][x][0][2] + discount_factor * V[env.P[id_s][x][0][1]]) for x in range(4)]
            V1[id_s] = np.max(lst)
            delta = max(delta, abs(V1[id_s] - V[id_s]))
#         print("delta ", delta)
        if (delta < theta):
            break
        else:
            V = V1
    return V1

def policy_improvment(env, V, policy):
    policy_prime = []
    print(V.reshape((4,4)))
    for id_s, state in enumerate(V):
        p1 = [0,0,0,0]
        next_states = [env.P[id_s][k][0][1] for k in range(4)]
#             next_nodes = [v[x[0][1]] for x in env.P[id_s].values()]
        max_s = np.argmax([V[n] for n in next_states])
#             print([V[n] for n in next_states])
        p1[max_s] = 1
        policy_prime.append(p1)
    return policy_prime

def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.        
    """

    V = np.zeros(env.nS)
    
    z = 0
    policy = np.ones([env.nS, env.nA]) / 4
    #policy evaluation
    V = policy_evaluation(env, discount_factor,theta,  V, policy)
    #policy iteration
    policy = policy_improvment(env, V, policy)
    # Implement!
    return policy, V

In [27]:
policy, v = value_iteration(env)

print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Policy Probability Distribution:
[[1, 0, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 1, 0, 0], [1, 0, 0, 0]]

Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):
[[0 3 3 2]
 [0 0 0 2]
 [0 0 1 2]
 [0 1 1 0]]

Value Function:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

Reshaped Grid Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]



In [28]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, 
                       -1, -2, -3, -2, 
                       -2, -3, -2, -1, 
                       -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)